## Combinators

Biological systems achieve much of their adaptability through the use of very general parts (cells) that are dynamically configured and consequently able to adjust as their environment changes. 

Computational systems usually do not use this strategy, instead relying on a hierarchy of custom parts and combinations. 

In recent years, large libraries of well-specified higher-level parts have raised the abstraction level of this activity. But the means of combination are rarely abstracted or shared, other than as "patterns".

In some situations we can improve on theis practice by simple strategies that promote the use of shared combination mechanisms. If the systems we build are made up from members of a family of "mix-and-match" components that combine to make new members of the family, perturbations of the requirements can sometimes be addressed by rearrangement of components.

A _system of combinators_ is a set of primitive parts and a set of means of combining parts such that the interface specifications of the combinations are the same as those of the primitives. This enables construction without accidental interactions between the parts.

Combinator systems provide a design strategy for domain-specific languages. The elements of the system are words in the language, and the combinators are used to combine them into phrases. Combinator systems have the significant advantage that they are easy to build and to reason about, but they have limitations, we which will discuss in _the next section_. When they fit the domain they are an excellent strategic choice.

But how do we arrange to buid our systems by combining elements of a family of mix-and-match components? We must identify a set of promitive components and a set of _combinators_ that combine components so as to make compound components with the same interface as the primitive components.

### Functional Combinators

The use of functional notation in mathematics is a combinator discipline. A function has a domain, from which its arguments are selected, and a range (or co-domain) of its possible values. There are combinators that produce new functions as combinations of others. For example the composition f &compfn; g of functions f and g is a new function that takes arguments in the domain of g and produces values in the codomain of f.

If two functions have the same domain and codomain, and if arithmetic is defined on their common codomain, then we can define the sum (or product) of the functions as the function that, when given an argument in their common domain, is the sum (or product) of the values of the two functions at that argument. Languages that allow first-class procedures provide a mechanism to support this means of combination, but what really matters is a good family of pieces.

Organizing a system around combinators has several advantages. The parts taht are made can be arbitrarily mixed and matched. Any combination yields a legal programme, whose behaviour transparently depends only on the behaviours of the parts and the way that they are combined. The context in whch a part appears does not change the behaviour of the part: it is always acceptable to pick up a component part to use it in a new context, without worry about its behaviour in that context. Thus such programmes are easy to write, easy to read, and easy to verify. _A programme built on combinators is extensible, because introduction of new parts or new combinators does not affect the behaviour of existing programmes._

We can think of function combinators as implementing wiring diagrams that specify how a function is built by combining its parts. For example, functional composition represents a box made of two sub-boxes so taht they output of the first feeds the input of the second. 

A programme that implements this idea is straightforward:

In [1]:
(defn compose [f g]
    (fn [& args]
        (f (apply g args))))

#'user/compose

In [2]:
((compose #(* % %) inc) 2)

9

(It gets more exciting if we want to check that the arities match: that the function represented by procedure f takes only one argument, to match the output of g. It gets even more fun if g can return multiple values and f must take those arguments. We may also want to check that the arguments passed to the composition are the right number for g. But these are fine points that awe will deal with later.)

We can demonstrate composition wiht a simple example:

In [3]:
((compose (fn [x] (list 'foo x))
          (fn [x] (list 'bar x)))
     'z)

(foo (bar z))